In [ ]:
# Перевірка підключення GPU
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.layers import Input, Dense, Layer, Dropout, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision, Recall


from transformers import TFBertForSequenceClassification, TFBertModel

import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.model_selection import train_test_split

**Підготовка кастомних класів і функцій**

Даний блок необхідно копіювати при завантажені моделей

In [ ]:
# Кастомний шар для інтеграції з BERT
class BertLayer(Layer):
    def __init__(self, pretrained_model_name="bert-base-uncased", trainable=False, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        # Завантажуємо попередньо навчений BERT
        self.bert = TFBertModel.from_pretrained(pretrained_model_name)
        self.bert.trainable = trainable  # Заморожуємо або розморожуємо шари залежно від параметра trainable

    def call(self, inputs):
        # Вхідні дані: input_ids та attention_mask
        input_ids, attention_mask = inputs
        # Передаємо дані через BERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state  # Повертаємо тільки last_hidden_state

In [ ]:
'''
у якості метрики обрано Ф-1 у зв'язку із незбалансованістю класів. 
Підготуємо функцію для неї
'''

import tensorflow as tf
from tensorflow.keras import backend as K

def f1_metric(y_true, y_pred):
    # Преобразуем в бинарный формат для каждого класса
    y_true = K.cast(y_true, 'int32')
    y_pred = K.cast(K.greater_equal(y_pred, 0.5), 'int32')

    # Вычисляем точность (precision) и полноту (recall)
    true_positive = K.sum(K.cast(y_true * y_pred, 'float32'))
    false_positive = K.sum(K.cast((1 - y_true) * y_pred, 'float32'))
    false_negative = K.sum(K.cast(y_true * (1 - y_pred), 'float32'))

    precision = true_positive / (true_positive + false_positive + K.epsilon())
    recall = true_positive / (true_positive + false_negative + K.epsilon())

    # F1-score = 2 * (precision * recall) / (precision + recall)
    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())
    
    return f1

In [ ]:
'''
підготовка вагів для функції врат щоб врахувати незбалансованість вибірки
'''
# Підрахунок ваг для кожного класу
class_counts = {
    0: 15294,  # toxic
    1: 1595,   # severe_toxic
    2: 8449,   # obscene
    3: 478,    # threat
    4: 7877,   # insult
    5: 1405    # identity_hate
}

# Загальна кількість прикладів у вибірці
total_samples = 159571

# Вага для кожного класу буде пропорційною оберненому співвідношенню його частоти
class_weights = {}
for label, count in class_counts.items():
    # Вага класу розраховується як обернене відношення загальної кількості прикладів
    class_weights[label] = total_samples / count

# Нормалізація ваг класів, щоб їх сума була рівна 1
class_weights = {k: v / sum(class_weights.values()) for k, v in class_weights.items()}

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K

def weighted_f1_loss(y_true, y_pred, class_weights):
    """
    Кастомна функція втрат для оптимізації макро F1-міри з урахуванням ваг класів.
    
    Args:
        y_true: tf.Tensor, істинні мітки (розмірність [batch_size, num_classes]).
        y_pred: tf.Tensor, передбачення моделі (розмірність [batch_size, num_classes]).
        class_weights: dict, ваги класів (ключі - індекси класів, значення - ваги).

    Returns:
        tf.Tensor, значення функції втрат.
    """
    # Застосовуємо сигмоїду до передбачень, якщо вони ще не пройшли через активацію
    y_pred = K.sigmoid(y_pred)

    # Перетворення ваг класів у тензор
    class_weight_tensor = tf.constant([class_weights[i] for i in range(len(class_weights))], dtype=tf.float32)

    # Обчислення TP, FP, FN
    true_positives = K.sum(y_true * y_pred, axis=0)
    predicted_positives = K.sum(y_pred, axis=0)
    actual_positives = K.sum(y_true, axis=0)

    # Обчислення Precision та Recall для кожного класу
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (actual_positives + K.epsilon())

    # Обчислення F1 для кожного класу
    f1_per_class = 2 * (precision * recall) / (precision + recall + K.epsilon())

    # Застосування ваг класів
    weighted_f1 = f1_per_class * class_weight_tensor

    # Середнє значення макро F1
    macro_f1 = K.mean(weighted_f1)

    # Повернення від'ємного значення F1 як функції втрат (для мінімізації)
    return 1 - macro_f1

**Загальна підготовка даних**

In [ ]:
data_path = '/kaggle/input/dataset/train_data.csv'
df = pd.read_csv(data_path)

# список категорій:
LABEL_COLUMNS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

# Конвертація токенізованих даних з рядків у масиви
for column in ['input_ids', 'attention_masks']:
    df[column] = df[column].apply(eval).apply(np.array)

# Виділяємо токенізовані вектори та мітки
input_ids = np.stack(df['input_ids'].values)
attention_mask = np.stack(df['attention_masks'].values)
labels = np.array(df[LABEL_COLUMNS].values)
labels = labels.astype('float32')

In [ ]:
# Розділення на тренувальну та тестову вибірки (повний набір даних)

train_input_ids, val_input_ids, train_attention_mask, val_attention_mask, train_labels, val_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=42
)

**Бінарна модель**

In [ ]:
# Генерація міток для бінарної моделі
t_binary_labels = np.where(np.all(train_labels == 0, axis=1), 1, 0).astype('float32')
v_binary_labels = np.where(np.all(val_labels == 0, axis=1), 1, 0).astype('float32')

In [ ]:
# Балансування даних

from imblearn.over_sampling import SMOTE

t_features = np.hstack((train_input_ids, train_attention_mask))

smote = SMOTE(random_state=42)
t_data_resampled, t_binary_labels_resampled = smote.fit_resample(t_features, t_binary_labels)

In [ ]:
# До SMOTE
print("До обробки:")
print(f"Токсичні: {np.sum(t_binary_labels == 0)}")
print(f"Нетоксичні: {np.sum(t_binary_labels == 1)}")

# Після SMOTE
print("\nПісля обробки:")
print(f"Токсичні: {np.sum(t_binary_labels_resampled == 0)}")
print(f"Нетоксичні: {np.sum(t_binary_labels_resampled == 1)}")

In [ ]:
# Зворотне перетворення на train_input_ids і t_attention_mask

# Вихідні розміри train_input_ids и train_attention_mask
input_ids_size = train_input_ids.shape[1]
attention_mask_size = train_attention_mask.shape[1]

# Зворотній розподіл
t_input_ids_resampled = t_data_resampled[:, :input_ids_size]
t_attention_mask_resampled = t_data_resampled[:, input_ids_size:]

In [ ]:
t_data = {
    "input_ids": t_input_ids_resampled,
    "attention_mask": t_attention_mask_resampled,
}

v_data = {
    "input_ids": val_input_ids,
    "attention_mask": val_attention_mask,
}

In [ ]:
# Модель для бінарної класифікації

# Вхідні дані
input_ids = Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(128,), dtype=tf.int32, name="attention_mask")

# BERT шар
bert_outputs = BertLayer(trainable=False)([input_ids, attention_mask])

# Пулінг
pooled_output = GlobalAveragePooling1D()(bert_outputs)

# бінарна класифікація
binary_dense = Dense(128, activation="swish")(pooled_output)
binary_dropout = Dropout(0.3)(binary_dense)
binary_output = Dense(1, activation="sigmoid", name="binary_output")(binary_dropout)

# Модель
model_3_1 = Model(
    inputs=[input_ids, attention_mask],
    outputs=[binary_output]
)

# Компіляція моделі
model_3_1.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',        
    patience=3,                
    restore_best_weights=True  
)

# Навчання моделі
history_3_1 = model_3_1.fit(
    t_data,
    t_binary_labels_resampled,
    validation_data=(v_data, v_binary_labels),
    epochs=10,  
    batch_size=128,
    callbacks=[early_stopping]  
)

In [ ]:
import json

# Збереження історії навчання в JSON
with open('history_3_1.json', 'w') as json_file:
    json.dump(history_3_1.history, json_file)


In [ ]:
model_3_1.save("model_3_1.h5")

In [ ]:
# Підготовка частини валідаціної вибірки для прогнозів 
_, test_input_ids, _, test_attention_mask, _, test_labels = train_test_split(
    val_input_ids, val_attention_mask, val_labels, test_size=0.1, random_state=42
)

test_binary_labels = np.where(np.all(test_labels == 0, axis=1), 1, 0).astype('float32')

In [ ]:
# Отримання прогнозів для першої моделі
binary_predictions = model_3_1.predict(
    {'input_ids': test_input_ids, 'attention_mask': test_attention_mask},
    batch_size=64
)

binary_predictions = (binary_predictions > 0.5).astype(int)  # Перетворення в 0 або 1

# Розподіл прогнозів бінарної моделі
unique, counts = np.unique(binary_predictions, return_counts=True)
binary_distribution = dict(zip(unique, counts))

print("Розподіл міток:")
print(f"Нетоксичні (1): {binary_distribution.get(1, 0)}")
print(f"Токсичні (0): {binary_distribution.get(0, 0)}")

In [ ]:
# Розподіл істинних міток
unique, counts = np.unique(test_binary_labels, return_counts=True)
true_distribution = dict(zip(unique, counts))

print("Розподіл істинних міток:")
print(f"Нетоксичні (1): {true_distribution.get(1, 0)}")
print(f"Токсичні (0): {true_distribution.get(0, 0)}")

In [ ]:
# Оцінка моделі
print("\nКласифікаційний звіт для бінарної моделі:\n")
print(classification_report(test_binary_labels, binary_predictions, target_names=[
    "non_toxic", "toxic"
]))

**Фінтюнінг бінарної моделі**

In [ ]:
# Розморозка останніх 4-х шарів BERT
for layer in model_3_1.layers:
    if isinstance(layer, BertLayer):  
        for bert_layer in layer.bert.bert.encoder.layer[-4:]:  
            bert_layer.trainable = True

model_3_1.compile(
    optimizer=Adam(learning_rate=1e-5),  # Низкий learning rate для фінтюнинга
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
# Финтюнинг
history_3_1 = model_3_1.fit(
    t_data,
    t_binary_labels_resampled,
    validation_data=(v_data, v_binary_labels),
    epochs=3,               # Невелика кількість епох, оскільки БЕРТ має навчатись дуже швидко
    batch_size=128
)

In [ ]:
# Збереження історії змін в JSON
with open('history_3_1_fin.json', 'w') as json_file:
    json.dump(history_3_1.history, json_file)

In [ ]:
model_3_1.save("model_3_1_fin.h5")

In [ ]:
# Отримання прогнозів для першої моделі
binary_predictions = model_3_1.predict(
    {'input_ids': test_input_ids, 'attention_mask': test_attention_mask},
    batch_size=64
)

binary_predictions = (binary_predictions > 0.5).astype(int)  # Перетворення в 0 або 1

# Розподіл прогнозів бінарної моделі
unique, counts = np.unique(binary_predictions, return_counts=True)
binary_distribution = dict(zip(unique, counts))

print("Розподіл міток:")
print(f"Нетоксичні (1): {binary_distribution.get(1, 0)}")
print(f"Токсичні (0): {binary_distribution.get(0, 0)}")

In [ ]:
# Оцінка моделі
print("\nКласифікаційний звіт для бінарної моделі:\n")
print(classification_report(test_binary_labels, binary_predictions, target_names=[
    "non_toxic", "toxic"
]))

**Багатоміткова модель**

У якості основи для багатоміткової моделі буде взята модель №2 із даного проекту - https://github.com/T-Dzv/toxic_finder/blob/dzv-model-4/model-2.ipynb

Ця модель хоч продемонструвала схильність до визначення більшості коментарів, як токсичних. Проте попри свої недоліки вона дійсно намагалась прогнозувати всі класи токсичності, навіть якщо її результати поки далекі від ідеальних. 

Навіть при поточних результатах модель дає не нульовий recall для всіх класів, у тому числі рідких, щого не вдалось добитись у інших спробах (перша частина модулю)

Використання цієї моделі в пайплайні разом із бінарною моделлю та із донавчанням (фінтюнінг на вибірці лише із токсичних коментарів) має потенціал. 

In [ ]:
# виділення із вибірки лише токсичних коментарів

# Видбірка токсичних коментарів
t_toxic_indices = np.any(train_labels == 1, axis=1)
v_toxic_indices = np.any(val_labels == 1, axis=1)

# Вхідні дані лише для токсичних прикладів
t_toxic_input_ids = train_input_ids[t_toxic_indices]
t_toxic_attention_mask = train_attention_mask[t_toxic_indices]
t_toxic_labels = train_labels[t_toxic_indices]

v_toxic_input_ids = val_input_ids[v_toxic_indices]
v_toxic_attention_mask = val_attention_mask[v_toxic_indices]
v_toxic_labels = val_labels[v_toxic_indices]

In [ ]:
# Завантаження попердньо навченої моделі
from keras.models import load_model

model_path = '/kaggle/input/pretrained/model.h5'
model_3_2 = load_model(model_path, custom_objects={'BertLayer': BertLayer}, compile=False)

In [ ]:
model_3_2.compile(
    optimizer=Nadam(learning_rate=1e-4),
    loss=lambda y_true, y_pred: weighted_f1_loss(y_true, y_pred, class_weights),
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), f1_metric]
)

**Фінтюнінг багатоміткової моделі**

In [ ]:
# Розморозка останніх 4-х шарів BERT
for layer in model_3_1.layers:
    if isinstance(layer, BertLayer):  
        for bert_layer in layer.bert.bert.encoder.layer[-4:]:  
            bert_layer.trainable = True

model_3_2.compile(
    optimizer=Nadam(learning_rate=1e-4),
    loss=lambda y_true, y_pred: weighted_f1_loss(y_true, y_pred, class_weights),
    metrics=["accuracy", Precision(name="precision"), Recall(name="recall"), f1_metric]
)

In [ ]:
# Навчання моделі
history_3_2 = model_3_2.fit(
    {
        'input_ids': t_toxic_input_ids,
        'attention_mask': t_toxic_attention_mask
    },
    t_toxic_labels,
    validation_data=(
        {
            'input_ids': v_toxic_input_ids,
            'attention_mask': v_toxic_attention_mask
        },
        v_toxic_labels),
    epochs=3, 
    batch_size=128
)

In [ ]:
# Збереження історії змін в JSON
with open('history_3_2_fin.json', 'w') as json_file:
    json.dump(history_3_2.history, json_file)

In [ ]:
model_3_2.save("model_3_2_fin.h5")

In [ ]:
# Прогнози мультиміткової моделі на валідаціних даних
multilabel_predictions = model_3_2.predict(
    {'input_ids': v_toxic_input_ids, 'attention_mask': v_toxic_attention_mask},
    batch_size=64
)

# Перетворюємо прогнози на бінарні мітки
multilabel_predictions = (multilabel_predictions > 0.5).astype(int)
# Сумуємо значення для кожної мітки
toxic_label_counts = multilabel_predictions.sum(axis=0)

# Мітки токсичності
labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

print("\nРозподіл міток багатоміткової моделі:")
for i, label in enumerate(labels):
    print(f"{label}: {toxic_label_counts[i]}")

In [ ]:
# Істиний розподіл міток
true_label_counts = v_toxic_labels.sum(axis=0)

print("\nРозподіл істиних міток:")
for i, label in enumerate(labels):
    print(f"{label}: {true_label_counts[i]}")

In [ ]:
# Оцінка моделі
print("\nКласифікаційний звіт для другого підходу моделі:\n")
print(classification_report(v_toxic_labels, multilabel_predictions, target_names=[
    "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"
]))

**Побудова загального пайплайну прогнозів**

In [ ]:
# Додання класу нетоксичних коментарів до міток тестової вибірки (всі нулі)
all_zeros_class = np.all(test_labels == 0, axis=1).astype(int)  
y_test_expanded = np.hstack((test_labels, all_zeros_class.reshape(-1, 1)))  

# Отримання прогнозів для першої моделі
binary_predictions = model_3_1.predict(
    {'input_ids': test_input_ids, 'attention_mask': test_attention_mask},
    batch_size=64
)
binary_predictions = (binary_output > 0.5).astype(int)  # Перетворення в 0 або 1

final_predictions = []
# Проходимо по кодному прикладу даних
for i in range(len(test_input_ids)):
    binary_prediction = binary_predictions[i]  # Прогноз бінарної моделі для поточного приклада

    if binary_prediction == 1:
        # Якщо коментар не токсичний, формуємо фінальний вектор
        final_predictions.append([0, 0, 0, 0, 0, 0, 1])  # Всі нулі + 1 на останьому індексі
    else:
        # Якщо коментар токсичний, формуємо прогноз мультимітковою моделлю
        toxic_input_ids = test_input_ids[i].reshape(1, -1)  # Приклад в форматі (1, 128)
        toxic_attention_mask = test_attention_mask[i].reshape(1, -1)

        # Прогноз мультимітковою моделлю
        multilabel_prediction = model_3_2.predict(
            {'input_ids': toxic_input_ids, 'attention_mask': toxic_attention_mask},
            batch_size=1
        )

        # Перетворення прогнозів
        multilabel_result = (multilabel_prediction > 0.5).astype(int).flatten().tolist()
        multilabel_result.append(0)  # Дадаємо 0 в останній індекс 

        # Додаємо результат у фінальні прогнози
        final_predictions.append(multilabel_result)

# Перетворення фінальних прогнозів в numpy-масив
final_predictions = np.array(final_predictions)

In [ ]:
# Оцінка моделі
print("\nКласифікаційний звіт для другого підходу моделі:\n")
print(classification_report(y_test_expanded, final_predictions, target_names=[
    "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate", "non_toxic"
]))

# Побудова багатоміткової матриці помилок
conf_matrices = multilabel_confusion_matrix(y_test_expanded, final_predictions)

# Приклад виводу (наприклад для "toxic")
print("Confusion matrix for 'toxic':")
print(conf_matrices[0])